# Aufgabe 6 (Scala: Kantorowitsch-Bäume)

_Verwenden Sie ausschließlich aus der Lehrveranstaltung bekannte Scala-Konstrukte, dabei sind `var` und seiteneffektbehaftete Methoden/Klassen/Objekte strikt untersagt._

<div class="alert alert-block alert-danger">
<b>WICHTIG:</b> Die <b>ursprüngliche</b> Klausur hat <b>damals</b> systembedingt <code>Stream</code> (<i>"Strom"</i>) statt <code>LazyList</code> verwendet!<br/>
Grund für die Abweichung <b>hier</b>: <i>"type Stream in package scala is deprecated since 2.13.0: Use LazyList instead of Stream"</i><br/>
<b>Ersetzen</b> Sie daher in der Aufgabenstellung die Begriffe <i>"Strom"</i> bzw. "<code>Stream</code>" <b>gedanklich</b> durch die <b>neuere</b> <code>LazyList</code>.
</div>

<img src="lib/2017_08_08_klausur_p10_Kantorowitsch.png" align="right" width="150px"/>
<i>Kantorowitsch-Bäume</i> dienen zur symbolischen Darstellung von Termen. Innere Knoten repräsentieren Operatoren, deren Operanden jeweils in den Unterbäumen stehen, während Blätter Variablen sind. Das nebenstehende Beispiel zeigt den Term $(a \vee b) \wedge \lnot c$ (Infix-Notation). Gegeben seien folgende Datentypen für Variablennamen bzw. für Baumknoten sowie die beiden Hilfsfunktionen:

In [ ]:
type Name = Char

abstract class KTree
case class And(l: KTree, r: KTree) extends KTree
case class Or(l: KTree, r: KTree) extends KTree
case class Not(c: KTree) extends KTree
case class Var(n: Name) extends KTree

// wertet KTree kt mit Variablenbelegung va aus
def eval: KTree => List[(Name, Boolean)] => Boolean = kt => va => kt match { // ...
   case And(l, r) => eval(l)(va) && eval(r)(va)
   case Or(l, r) => eval(l)(va) || eval(r)(va)
   case Not(c) => !eval(c)(va)
   case Var(n) => va.unzip._2(va.unzip._1.indexOf(n))
}

// liefert Liste aller Variablen im KTree
def vars: KTree => List[Name] = { // ...
   case And(l, r) => (vars(l) ::: vars(r)).distinct
   case Or(l, r) => (vars(l) ::: vars(r)).distinct
   case Not(c) => vars(c)
   case Var(n) => List(n)
}

**a)** Ergänzen Sie die Funktion `all` so, dass sie für _alle_ Variablen aus der Eingabeliste den Strom aller möglichen Belegungen generiert. Verwenden Sie _**<ins>unbedingt</ins>**_ eine `for`-comprehension mit **zwei** Generatoren, jedoch _**<ins>keine</ins>**_ `Stream`- oder `List`-Konkatenation (`::` und `#::` sind erlaubt):

**Beispiel:**
```scala
  all(List('a', 'b')).toList == List(List(('a', false), ('b', false)),
      List(('a', false), ('b', true)), List(('a', true), ('b', false)),
      List(('a', true), ('b', true))) 
```

In [ ]:
def all: List[Name] => LazyList[List[(Name, Boolean)]] = {
   case Nil => LazyList(Nil)
   case v :: vs =>
      /*** IHR CODE HIER ***/
}

In [ ]:
// Beispiel und Schnelltest:
{
def check: List[List[(Name, Boolean)]] => List[List[(Name, Boolean)]] => Boolean = expected => actual =>
   actual.distinct == actual &&
   expected.length == actual.length &&
   (for(p <- expected.map(_.sortBy(_._1))) yield actual.map(_.sortBy(_._1)).contains(p)).forall(b=>b) &&
   (for(p <- actual.map(_.sortBy(_._1))) yield expected.map(_.sortBy(_._1)).contains(p)).forall(b=>b)

assert(check(List(List(('a', false), ('b', false)),
                  List(('a', false), ('b', true)), List(('a', true), ('b', false)),
                  List(('a', true), ('b', true))))
       (all(List('a', 'b')).toList)); print("✅")
assert(check(List(List(('a', false), ('b', false)),
                  List(('a', false), ('b', true)), List(('a', true), ('b', false)),
                  List(('a', true), ('b', true))))
       (all(List('b', 'a')).toList)); print("✅")
print("|")
assert(check(List(List(('x', false)), List(('x', true)))) (all(List('x')).toList)); print("✅")
assert(check(List(List(('x', false), ('y', false), ('z', false)), 
                  List(('x', false), ('y', false), ('z', true)),
                  List(('x', false), ('y', true), ('z', false)),
                  List(('x', false), ('y', true), ('z', true)),
                  List(('x', true), ('y', false), ('z', false)),
                  List(('x', true), ('y', false), ('z', true)),
                  List(('x', true), ('y', true), ('z', false)),
                  List(('x', true), ('y', true), ('z', true))))
       (all(List('x', 'y', 'z')).toList)); print("✅")
}

**b)** Ergänzen Sie die folgende Funktion so, dass sie für den übergebenen _Kantorowitsch-Baum_ den Strom aller möglichen Variablenbelegungen generiert, für die der Baum zu `true` ausgewertet werden würde - verwenden Sie dazu _**<ins>unbedingt</ins>**_ `filter`:

In [ ]:
def sat: KTree => LazyList[List[(Name, Boolean)]] = {
   /*** IHR CODE HIER ***/
}

In [ ]:
// Beispiel und Schnelltest:
{
def check: List[List[(Name, Boolean)]] => List[List[(Name, Boolean)]] => Boolean = expected => actual =>
   actual.distinct == actual &&
   expected.length == actual.length &&
   (for(p <- expected.map(_.sortBy(_._1))) yield actual.map(_.sortBy(_._1)).contains(p)).forall(b=>b) &&
   (for(p <- actual.map(_.sortBy(_._1))) yield expected.map(_.sortBy(_._1)).contains(p)).forall(b=>b)

assert(sat(Var('x')) == LazyList(List(('x', true)))); print("✅")
assert(check(List(List(('x', true)))) (sat(Var('x')).toList)); print("✅")
print("|")
assert(check(List(List(('x', true), ('y', true)))) (sat(And(Var('x'),Var('y'))).toList)); print("✅")
print("|")
assert(check(List(List(('x', true), ('y', true)), List(('x', true), ('y', false)), List(('y', true), ('x', false))))
       (sat(Or(Var('x'),Var('y'))).toList)); print("✅")
print("|")
assert(check(List(List(('x', false)))) (sat(Not(Var('x'))).toList)); print("✅")
print("|")
var example = And(Or(Var('a'),Var('b')),Not(Var('c')))
assert(check(List(
    List(('a', true), ('b', true), ('c', false)),
    List(('a', true), ('b', false), ('c', false)),
    List(('a', false), ('b', true), ('c', false))))
       (sat(example).toList)); print("✅")
}

**c)** Ergänzen Sie schließlich die folgende Funktion, die die übergebene Liste von Bäumen mit dem `And`-Operator verknüpft und zu einem Baum zusammensetzt. Verwenden Sie _**<ins>unbedingt</ins>**_ `foldLeft` statt Rekursion:

In [ ]:
def and: List[KTree] => KTree = {
   /*** IHR CODE HIER ***/
   case ls => ??? // undefiniert
}

In [ ]:
// Beispiel und Schnelltest:
assert(and(List(Var('x'))) == Var('x')); print("✅")
assert(and(List(Var('x'),Var('y'))) == And(Var('x'),Var('y'))); print("✅")
assert(and(List(Var('a'),Var('b'),Var('c'))) == And(And(Var('a'),Var('b')),Var('c'))); print("✅")
assert(and(List(Var('a'),Var('b'),Var('c'),Var('d'))) == And(And(And(Var('a'),Var('b')),Var('c')),Var('d'))); print("✅")
{
var example = And(Or(Var('a'),Var('b')),Not(Var('c')))
var anotherExample = Or(And(Var('d'),Var('e')),Not(Var('f')))
var yetAnotherExample = And(Not(Var('g')),And(Var('h'),Var('i')))
assert(and(List(example,anotherExample,yetAnotherExample)) == And(And(example,anotherExample),yetAnotherExample)); print("✅")
}